# 04 - Race Program: Model Deployment
Author: George Gorospe, george.gorospe@nmaia.net (updated 1/17/2024)

# In this fourth notebook, we'll use the the machine learning model we trained previously to pilot our vehicle around a track.

We've done a lot of work collecting images and training a model to act as the pilot of our self-driving race car. 
In this notebook, we'll input the live camera feed into the model and take steering directions as our output.s our pilot for our self-driving car.

In [1]:
# Importing Required Libraries

# Machine Learning Libraries
import torch
from torch2trt import TRTModule
import torchvision.transforms as transforms
import torch.nn.functional as F

# Vehicle Control Libraries from invidia
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg


# General Use Libraries
import cv2
import ipywidgets
import threading
import PIL.Image
import numpy as np


# Jupyter Labs Libraries
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets
from ipyfilechooser import FileChooser


# Useful Variables
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

# Custom functions
def preprocess(image):
    device = torch.device('cuda')
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

## Starting Vehicle Control and Camera Control

In [2]:
camera = CSICamera(width=224, height=224)
car = NvidiaRacecar()

GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3280 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3280 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: Running with following settings:
   Camera index = 0 
   Camera mode  = 4 
   Output Stream W = 1280 H = 7

[ WARN:0] global /home/ubuntu/build_opencv/opencv/modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


## Selecting a trained Model
Use the file chooser to select your model

In [3]:
# Create and display a FileChooser widget
fc = FileChooser('/home/racer_core/Models/trt')
display(fc)
fc.filter_pattern = ['*.pth']
# Change the title (use '' to hide)
fc.title = '<b>Choose Model for Race Program </b>'

# Sample callback function
def change_title(chooser):
    chooser.title = '<b>Model Selected.</b>'

# Register callback function
fc.register_callback(change_title)

FileChooser(path='/home/racer_core/Models/trt', filename='', title='', show_hidden=False, select_desc='Select'…

## Creating a Graphical User Interface for Race Program

In [4]:
# Warm starting the optimized model with the weights from our trained and optimized model saved in last notebook
model_trt = TRTModule()
model_trt_path = fc.selected
model_trt.load_state_dict(torch.load(model_trt_path))

# Creating a Driving/Prediction Graphical User Interface
state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-1.0, max=2.0, value=1.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0.15, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([steering_gain_slider, steering_bias_slider],layout=Layout(align_items='center')),
         live_execution_widget,
         throttle_slider]
    )
)

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[0])
    
    network_output_slider.value = x
    steering = x * steering_gain_slider.value + steering_bias_slider.value
    if(steering<-1.0):
        steering_value_slider.value = -1.0
    elif(steering>1.0):
        steering_value_slider.value = 1.0
    else:
        steering_value_slider.value = steering 

    # Sending steering value to car
    car.steering = steering # Previously was x
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))
        steering_mapping = int(camera.width * (steering / 2.0 + 0.5))
        prediction = new_image.copy()
        # Plotting the output of the model in GREEN
        #prediction[0:90, 0:224] = 0
        prediction = cv2.circle(prediction, (x, 112), 8, (255, 0, 0), 3)
        # Plotting the car steering command in BLUE
        prediction_and_steering = cv2.circle(prediction, (steering_mapping, 112), 8, (0, 255, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction_and_steering)
        # ADD STEERING WIDGET VALUE HERE
        
update({'new': camera.value})  # we call the function once to initialize

[11/15/2024-20:11:27] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.


 # <font color='BLUE'>BLUE = Model Output</font> &nbsp;&nbsp;&nbsp;&nbsp; <font color='GREEN'>Green = Car Steering Command</font> 
 Run the next two cells to start the camera.
 When you're done racing, run the final cell to stop the camera.

 


In [5]:
camera.observe(update, names='value') 

In [6]:
camera.running = True

In [ ]:
camera.close